In [0]:
# Write the sample files.

# A regular JSON file.
dbutils.fs.put("/tmp/test1.json", 
'''{ "Text1":"hello", "Text2":"goodbye", "Num1":5, "Array1":[7,8,9] }
{ "Text1":"this", "Text2":"that", "Num1":6.6, "Array1":[77,88,99] }
{ "Text1":"yes", "Text2":"no", "Num1":-0.03, "Array1":[555,444,222] }''', 
True)

# A text file that contains one JSON field. 
# We use vertical bar as the field separator because that eliminates problems with quotes and commas inside the JSON.
dbutils.fs.put("/tmp/test2.txt", 
'''Text1|Text2|Num1|JSON1
hello | goodbye | 5 | {"Sub1":"john", "Sub2":3}
this | that | 6.6 | {"Sub1":"betty", "Sub2":4}
yes | no | -0.03 | {"Sub1":"bobby", "Sub2":5}''',
True)

# A text file that contains one JSON field that is an array of objects. 
dbutils.fs.put("/tmp/test3.txt", 
'''Text1|Text2|Num1|JSON1
hello | goodbye | 5 | [{"Sub1":"stop", "Sub2":3}, {"Sub1":"go", "Sub2":6}]
this | that | 6.6 | [{"Sub1":"eggs", "Sub2":4}, {"Sub1":"bacon", "Sub2":8}]
yes | no | -0.03 | [{"Sub1":"apple", "Sub2":5}, {"Sub1":"pear", "Sub2":10}]''',
True)



Wrote 204 bytes.
Wrote 163 bytes.
Wrote 249 bytes.
Out[1]: True

In [0]:
# Verify the presence of the files in the Databricks file system.

dbutils.fs.ls("/tmp/")

Out[2]: [FileInfo(path='dbfs:/tmp/hive/', name='hive/', size=0),
 FileInfo(path='dbfs:/tmp/test1.json', name='test1.json', size=204),
 FileInfo(path='dbfs:/tmp/test2.txt', name='test2.txt', size=163),
 FileInfo(path='dbfs:/tmp/test3.txt', name='test3.txt', size=249)]

In [0]:
# Show the file that contains plain JSON.

dbutils.fs.head("/tmp/test1.json")

Out[3]: '{ "Text1":"hello", "Text2":"goodbye", "Num1":5, "Array1":[7,8,9] }\n{ "Text1":"this", "Text2":"that", "Num1":6.6, "Array1":[77,88,99] }\n{ "Text1":"yes", "Text2":"no", "Num1":-0.03, "Array1":[555,444,222] }'

In [0]:
#Make a DataFrame from this file. PySpark will infer the data types of the JSON elements.

test1DF = spark.read.json("/tmp/test1.json")

# Show the DataFrame. Note that the data is no longer in JSON format. All the JSON syntax has been stripped out and we have regular data fields.

display (test1DF)

Array1,Num1,Text1,Text2
"List(7, 8, 9)",5.0,hello,goodbye
"List(77, 88, 99)",6.6,this,that
"List(555, 444, 222)",-0.03,yes,no


In [0]:
# Show the file with regular fields and one field with JSON text.

dbutils.fs.head("/tmp/test2.txt")

Out[5]: 'Text1|Text2|Num1|JSON1\nhello | goodbye | 5 | {"Sub1":"john", "Sub2":3}\nthis | that | 6.6 | {"Sub1":"betty", "Sub2":4}\nyes | no | -0.03 | {"Sub1":"bobby", "Sub2":5}'

In [0]:
# Make a DataFrame from the text file that contains a JSON field.

# This is a small file, so we can infer the schema. For a large file, you should use .schema().
test2DF = spark.read.option("inferSchema", True).option("header", True).option("delimiter", "|").csv("/tmp/test2.txt")

# Show the DataFrame. Note that the JSON field is plain text; we cannot pick out its sub-fields.
display (test2DF)

Text1,Text2,Num1,JSON1
hello,goodbye,5.0,"{""Sub1"":""john"", ""Sub2"":3}"
this,that,6.6,"{""Sub1"":""betty"", ""Sub2"":4}"
yes,no,-0.03,"{""Sub1"":""bobby"", ""Sub2"":5}"


In [0]:
# Change the JSON string into a struct, so we can get its parts. Then pull out one of those parts.

from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Define the schema of the JSON string.
schema = StructType([
  StructField("Sub1", StringType()), 
  StructField("Sub2", IntegerType()
  )
])

# Use the schema to change the JSON string into a struct, overwriting the JSON string.
test2DF = test2DF.withColumn("JSON1", from_json(col("JSON1"), schema))

# Make a separate column from one of the struct fields.
test2DF = test2DF.withColumn("JSON1_Sub2", col("JSON1.Sub2"))

display (test2DF)

Text1,Text2,Num1,JSON1,JSON1_Sub2
hello,goodbye,5.0,"List(john, 3)",3
this,that,6.6,"List(betty, 4)",4
yes,no,-0.03,"List(bobby, 5)",5


In [0]:
# Show the file that contains an array of JSON objects.

dbutils.fs.head("/tmp/test3.txt")

Out[10]: 'Text1|Text2|Num1|JSON1\nhello | goodbye | 5 | [{"Sub1":"stop", "Sub2":3}, {"Sub1":"go", "Sub2":6}]\nthis | that | 6.6 | [{"Sub1":"eggs", "Sub2":4}, {"Sub1":"bacon", "Sub2":8}]\nyes | no | -0.03 | [{"Sub1":"apple", "Sub2":5}, {"Sub1":"pear", "Sub2":10}]'

In [0]:
# Make a DataFrame from the text file that contains a JSON array.

test3DF = spark.read.option("inferSchema", True).option("header", True).option("delimiter", "|").csv("/tmp/test3.txt")

display (test3DF)

Text1,Text2,Num1,JSON1
hello,goodbye,5.0,"[{""Sub1"":""stop"", ""Sub2"":3}, {""Sub1"":""go"", ""Sub2"":6}]"
this,that,6.6,"[{""Sub1"":""eggs"", ""Sub2"":4}, {""Sub1"":""bacon"", ""Sub2"":8}]"
yes,no,-0.03,"[{""Sub1"":""apple"", ""Sub2"":5}, {""Sub1"":""pear"", ""Sub2"":10}]"


In [0]:
# Change the JSON field to a true array.

# Credit to https://kontext.tech/column/spark/284/pyspark-convert-json-string-column-to-array-of-object-structtype-in-data-frame

from pyspark.sql.functions import from_json, col, split, udf
from pyspark.sql.types import *
import json

# Schema for the array of JSON objects.
json_array_schema = ArrayType(
  StructType([
    StructField('Sub1', StringType(), nullable=False), 
    StructField('Sub2', IntegerType(), nullable=False)
  ])
)

# Create function to parse JSON using standard Python json library.
def parse_json(array_str):
  json_obj = json.loads(array_str)
  for item in json_obj:
    yield (item['Sub1'], item['Sub2'])

# Create a UDF, whose return type of the JSON schema defined above.    
parse_json_udf = udf(lambda str: parse_json(str), json_array_schema)
  
# Use the UDF to change the JSON string into a true array of objects.
test3DF = test3DF.withColumn("JSON1arr", parse_json_udf((col("JSON1"))))

# We don't need to JSON text anymore.
test3DF = test3DF.drop("JSON1")

# We now have an array of structs with named members.
display (test3DF)

Text1,Text2,Num1,JSON1arr
hello,goodbye,5.0,"List(List(stop, 3), List(go, 6))"
this,that,6.6,"List(List(eggs, 4), List(bacon, 8))"
yes,no,-0.03,"List(List(apple, 5), List(pear, 10))"


In [0]:
# The array of structs might be useful as is. But it is sometimes easier to put each array element in its own DataFrame row.

from pyspark.sql.functions import col, explode

test3DF = test3DF.withColumn("JSON1obj", explode(col("JSON1arr")))

# The column with the array is now redundant.
test3DF = test3DF.drop("JSON1arr")

display (test3DF)


Text1,Text2,Num1,JSON1obj
hello,goodbye,5.0,"List(stop, 3)"
hello,goodbye,5.0,"List(go, 6)"
this,that,6.6,"List(eggs, 4)"
this,that,6.6,"List(bacon, 8)"
yes,no,-0.03,"List(apple, 5)"
yes,no,-0.03,"List(pear, 10)"


In [0]:
# Get a specific named element from all the JSON objects. This is much easier now that we exploded the array.

display (test3DF.select("JSON1obj.Sub1"))

Sub1
stop
go
eggs
bacon
apple
pear
